In [442]:
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [443]:
path = "/content/drive/MyDrive/PyTorch/Data/shakespeare.txt"

In [444]:
with open(path,mode = "r", encoding = "utf8") as f:
    text = f.read()

In [445]:
text[:1000]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee.\n\n\n                     2\n  When forty winters shall besiege thy brow,\n  And dig deep trenches in thy beauty's field,\n  Thy youth's proud livery so gazed on now,\n  Will be a tattered weed of small worth held:  \n  Then being asked, where all thy beauty lies,\n  Where all the treasure of thy lusty days;\n  To sa

In [446]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [447]:
len(text)

5445609

In [448]:
all_characters = set(text)
#all_characters

In [449]:
len(all_characters)

84

In [450]:
decoder = dict(enumerate(all_characters))
decoder.items()

dict_items([(0, 'H'), (1, ' '), (2, 's'), (3, ']'), (4, 'T'), (5, 'a'), (6, 'O'), (7, 'C'), (8, 'b'), (9, '>'), (10, 'S'), (11, '('), (12, '2'), (13, '4'), (14, 't'), (15, 'y'), (16, 'i'), (17, 'd'), (18, ','), (19, 'R'), (20, 'x'), (21, 'v'), (22, '8'), (23, 'r'), (24, 'q'), (25, '.'), (26, 'h'), (27, '`'), (28, '"'), (29, 'G'), (30, '0'), (31, 'F'), (32, ')'), (33, '_'), (34, '\n'), (35, 'E'), (36, 'Y'), (37, 'g'), (38, 'U'), (39, '6'), (40, 'A'), (41, 'M'), (42, 'k'), (43, '|'), (44, '1'), (45, "'"), (46, '7'), (47, 'j'), (48, '3'), (49, 'Z'), (50, 'n'), (51, '<'), (52, 'c'), (53, 'N'), (54, 'z'), (55, '}'), (56, 'f'), (57, 'B'), (58, 'D'), (59, 'W'), (60, 'o'), (61, ';'), (62, 'J'), (63, 'V'), (64, ':'), (65, '['), (66, 'u'), (67, '-'), (68, 'K'), (69, 'I'), (70, '!'), (71, 'e'), (72, 'L'), (73, 'w'), (74, 'l'), (75, '&'), (76, '9'), (77, 'p'), (78, 'm'), (79, 'P'), (80, '?'), (81, 'Q'), (82, '5'), (83, 'X')])

In [451]:
encoder = {char : ind for ind,char in decoder.items()}
encoder.items()

dict_items([('H', 0), (' ', 1), ('s', 2), (']', 3), ('T', 4), ('a', 5), ('O', 6), ('C', 7), ('b', 8), ('>', 9), ('S', 10), ('(', 11), ('2', 12), ('4', 13), ('t', 14), ('y', 15), ('i', 16), ('d', 17), (',', 18), ('R', 19), ('x', 20), ('v', 21), ('8', 22), ('r', 23), ('q', 24), ('.', 25), ('h', 26), ('`', 27), ('"', 28), ('G', 29), ('0', 30), ('F', 31), (')', 32), ('_', 33), ('\n', 34), ('E', 35), ('Y', 36), ('g', 37), ('U', 38), ('6', 39), ('A', 40), ('M', 41), ('k', 42), ('|', 43), ('1', 44), ("'", 45), ('7', 46), ('j', 47), ('3', 48), ('Z', 49), ('n', 50), ('<', 51), ('c', 52), ('N', 53), ('z', 54), ('}', 55), ('f', 56), ('B', 57), ('D', 58), ('W', 59), ('o', 60), (';', 61), ('J', 62), ('V', 63), (':', 64), ('[', 65), ('u', 66), ('-', 67), ('K', 68), ('I', 69), ('!', 70), ('e', 71), ('L', 72), ('w', 73), ('l', 74), ('&', 75), ('9', 76), ('p', 77), ('m', 78), ('P', 79), ('?', 80), ('Q', 81), ('5', 82), ('X', 83)])

In [452]:
encoded_text = np.array([encoder[char] for char in text])
encoded_text[:500]

array([34,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 44, 34,  1,  1, 31, 23, 60, 78,  1, 56,  5, 16,
       23, 71,  2, 14,  1, 52, 23, 71,  5, 14, 66, 23, 71,  2,  1, 73, 71,
        1, 17, 71,  2, 16, 23, 71,  1, 16, 50, 52, 23, 71,  5,  2, 71, 18,
       34,  1,  1,  4, 26,  5, 14,  1, 14, 26, 71, 23, 71,  8, 15,  1,  8,
       71,  5, 66, 14, 15, 45,  2,  1, 23, 60,  2, 71,  1, 78, 16, 37, 26,
       14,  1, 50, 71, 21, 71, 23,  1, 17, 16, 71, 18, 34,  1,  1, 57, 66,
       14,  1,  5,  2,  1, 14, 26, 71,  1, 23, 16, 77, 71, 23,  1,  2, 26,
       60, 66, 74, 17,  1,  8, 15,  1, 14, 16, 78, 71,  1, 17, 71, 52, 71,
        5,  2, 71, 18, 34,  1,  1,  0, 16,  2,  1, 14, 71, 50, 17, 71, 23,
        1, 26, 71, 16, 23,  1, 78, 16, 37, 26, 14,  1,  8, 71,  5, 23,  1,
       26, 16,  2,  1, 78, 71, 78, 60, 23, 15, 64, 34,  1,  1, 57, 66, 14,
        1, 14, 26, 60, 66,  1, 52, 60, 50, 14, 23,  5, 52, 14, 71, 17,  1,
       14, 60,  1, 14, 26

# One Hot Encoding

In [453]:
test = np.array([1,2,3])
test, test.size

(array([1, 2, 3]), 3)

In [454]:
encoded_text.size,len(set(encoded_text))

(5445609, 84)

In [455]:
# Kaç tane sütun olmasını istediğimi gösteriyor !!!
num_uni_chars = 4
# test.size ise kaç tane satır olmaması belirliyor !!!

one_hot = np.zeros((test.size, num_uni_chars))
print(one_hot.shape)

one_hot = one_hot.astype(np.float32)
one_hot[ np.arange(one_hot.shape[0]), test.flatten()] = 1.0
print(one_hot.shape)
one_hot

(3, 4)
(3, 4)


array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [456]:
(*test.shape,num_uni_chars)

(3, 4)

In [457]:
one_hot = one_hot.reshape((*test.shape, num_uni_chars))
print(one_hot.shape)
one_hot

(3, 4)


array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [458]:
# aslında yapılan bu 
# Basit şekilde göstermeye çalışıyorum !!1
deneme = np.zeros((5,3)).astype(np.float32) 
deneme[ :deneme.shape[0], (1,2) ] = 1.0
deneme

array([[0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.]], dtype=float32)

In [459]:
def one_hot_encoder(encoded_text, num_uni_chars):

    """

    encoded_text = batch of encoded text

    num_uni_chars = number of unique characters (len(set(text)))

    """

    # create a placeholder for zeros
    one_hot = np.zeros((encoded_text.size, num_uni_chars))

    # Convert Data Type For Later Use With PyTorch (Errors If We Don't!)
    one_hot = one_hot.astype(np.float32)

    # Using fancy indexing fill in the 1s at correct index locations

    one_hot[ np.arange(one_hot.shape[0]), encoded_text.flatten() ] = 1.0

    one_hot = one_hot.reshape( (*encoded_text.shape, num_uni_chars))

    return one_hot

In [460]:
np.arange(10).reshape((5,2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [461]:
one_hot_encoder(np.array([1,2,0,1,4]) , 5)

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

# Creating Training Batches

In [462]:
example_text = np.arange(10)
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [463]:
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [464]:
def generate_batches(encoded_text, sample_per_batch = 10, seq_len = 50):

        
    '''
    Generate (using yield) batches for training.
    
    X: Encoded Text of length seq_len
    Y: Encoded Text shifted by one
    
    Example:
    
    X:
    
    [[1 2 3]]
    
    Y:
    
    [[ 2 3 4]]
    
    encoded_text : Complete Encoded Text to make batches from
    batch_size : Number of samples per batch
    seq_len : Length of character sequence
       
    '''

    # Total number of characters per batch
    # Example: If samp_per_batch is 2 and seq_len is 50, then 100
    # characters come out per batch.
    char_per_batch = sample_per_batch * seq_len

    # Number of batches avaiable to make
    # Use int() to roun to nearest integer
    num_batches_avail = int(len(encoded_text) / char_per_batch)
    
    # Cut off end of encoded_text that
    # won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]

    # Reshape text intp rows the size of a batch
    encoded_text = encoded_text.reshape((sample_per_batch, -1))


    # Go through each row in array.
    for n in range(0 , encoded_text.shape[1], seq_len):

        x = encoded_text[: , n:n+seq_len]
        y = np.zeros_like(x)

        try:

            y[: , :-1] = x[:, 1:]
            y[: , -1 ] = encoded_text[:, n+seq_len]

        # FOR POTENTIAL INDEXING ERROR AT THE END  
        except:
            
            y[:, :-1] = x[: , 1:]
            y[:, -1 ] = encoded_text[:, 0]

        yield x,y

In [465]:
for n in np.arange(10).reshape((5,2)):
    print(n)

[0 1]
[2 3]
[4 5]
[6 7]
[8 9]


In [466]:
sample_text = encoded_text[:100]
sample_text

array([34,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 44, 34,  1,  1, 31, 23, 60, 78,  1, 56,  5, 16,
       23, 71,  2, 14,  1, 52, 23, 71,  5, 14, 66, 23, 71,  2,  1, 73, 71,
        1, 17, 71,  2, 16, 23, 71,  1, 16, 50, 52, 23, 71,  5,  2, 71, 18,
       34,  1,  1,  4, 26,  5, 14,  1, 14, 26, 71, 23, 71,  8, 15,  1,  8,
       71,  5, 66, 14, 15, 45,  2,  1, 23, 60,  2, 71,  1, 78, 16])

In [467]:
batch_generator = generate_batches(sample_text, sample_per_batch=2, seq_len = 4)

x,y = next(batch_generator)

In [468]:
x

array([[34,  1,  1,  1],
       [ 1, 73, 71,  1]])

In [469]:
one_hot_encoder(x,84).shape

(2, 4, 84)

In [470]:
one_hot_encoder(x,84)[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

In [471]:
y

array([[ 1,  1,  1,  1],
       [73, 71,  1, 17]])

In [472]:
batch_generator = generate_batches(encoded_text, sample_per_batch=182, seq_len = 100)

x,y = next(batch_generator)

print(one_hot_encoder(x,84).shape)
print()
print(y.shape)

(182, 100, 84)

(182, 100)


In [473]:
x.size,(*x.shape,84)

(18200, (182, 100, 84))

In [474]:
y.shape

(182, 100)

# GPU Check

In [475]:
torch.cuda.is_available()

True

# Creating The LSTM Model

In [476]:
class CharModel(nn.Module):

    def __init__(self, all_chars, num_hidden = 256, num_layers = 4, drop_prob = 0.5, use_gpu = False):

        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu

        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars)) 
        self.encoder = {char : ind for ind, char in decoder.items()}

        self.lstm = nn.LSTM( input_size = len(self.all_chars), 
                            hidden_size = num_hidden, 
                            num_layers  = num_layers , 
                            dropout     = drop_prob,
                            batch_first = True)
        
        self.dropout = nn.Dropout(drop_prob)
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

    def forward(self, x, hidden):

        lstm_output, hidden = self.lstm(x, hidden)

        drop_output = self.dropout(lstm_output)

        drop_output = drop_output.contiguous().view(-1,self.num_hidden)

        final_out = self.fc_linear(drop_output)

        return final_out, hidden

    def hidden_state(self, batch_size):

        '''
        Used as separate method to account for both GPU and CPU users.
        '''

        if self.use_gpu :

            hidden = (torch.zeros(self.num_layers, batch_size , self.num_hidden).cuda(),
                      torch.zeros(self.num_layers, batch_size , self.num_hidden).cuda())
            
        else:

            hidden = (torch.zeros(self.num_layers, batch_size , self.num_hidden),
                      torch.zeros(self.num_layers, batch_size , self.num_hidden))
            
        return hidden

In [477]:
model = CharModel(
    all_chars = all_characters,
    num_hidden = 512,
    num_layers = 3,
    drop_prob = 0.5,
    use_gpu = True
)
model

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [478]:
total_params = []
for p in model.parameters():
    print(p.numel())
    total_params.append(int(p.numel()))

print(f"_____\n{sum(total_params)}")

172032
1048576
2048
2048
1048576
1048576
2048
2048
1048576
1048576
2048
2048
43008
84
_____
5470292


In [479]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [480]:
train_percent = 0.1
len(encoded_text)

5445609

In [481]:
int(len(encoded_text) * train_percent)

544560

In [482]:
train_ind = int(len(encoded_text) * train_percent)
train_ind

544560

In [483]:
train_percent = 0.1
train_ind = int(len(encoded_text) * train_percent)

train_data = encoded_text[:train_ind]
test_data = encoded_text[train_ind:]
train_data.shape, test_data.shape

((544560,), (4901049,))

# Training The Network

In [484]:
epochs = 50
batch_size = 128
seq_len = 100
tracker = 0
num_char = max(encoded_text) + 1

In [485]:
test = model.hidden_state(batch_size)
test[0].shape, test[1].shape

(torch.Size([3, 128, 512]), torch.Size([3, 128, 512]))

In [486]:
batch = generate_batches(train_data, batch_size, seq_len)
x,y = next(batch)

print(x)
print("----")
print(y)

[[34  1  1 ...  1 78 16]
 [15 18  1 ... 18 34  1]
 [16 50  5 ...  2 73 71]
 ...
 [19  6 10 ... 74  5 50]
 [23 18  1 ... 73 50 71]
 [ 5 23 14 ... 15 34  1]]
----
[[ 1  1  1 ... 78 16 37]
 [18  1  1 ... 34  1  1]
 [50  5 14 ... 73 71 71]
 ...
 [ 6 10 40 ...  5 50 17]
 [18  1  1 ... 50 71 23]
 [23 14  1 ... 34  1  1]]


In [487]:
model.train()

if model.use_gpu:
    model.cuda()

for i in range(epochs):

    hidden = model.hidden_state(batch_size)

    for x,y in generate_batches(train_data, batch_size, seq_len):

        tracker+=1

        x = one_hot_encoder(x, num_char)
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)

        if model.use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        hidden = tuple([state.data for state in hidden])
        
        model.zero_grad()
        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size * seq_len).long())

        loss.backward()

        # POSSIBLE EXPLODING GRADIENT PROBLEM!
        # LET"S CLIP JUST IN CASE
        nn.utils.clip_grad_norm_(model.parameters() , max_norm = 5)

        optimizer.step()

        ###################################
        ### CHECK ON VALIDATION SET ######
        #################################

        if tracker % 25 == 0:

            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()

            for x,y in generate_batches(val_data, batch_size, seq_len):

                x = one_hot_encoder(x, num_char)

                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                if model.use_gpu:

                    inputs = inputs.cuda()
                    targets = targets.cuda()

                val_hidden = tuple([state.data for state in val_hidden])

                lstm_output, val_hidden = model.forward(inputs , val_hidden)
                val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())

                val_losses.append(val_loss.item())

            model.train()

            print(f"Epoch : {i:3}, Step : {tracker:3}, Val Loss : {val_loss.item()}")

Epoch :   0, Step :  25, Val Loss : 3.214017868041992
Epoch :   1, Step :  50, Val Loss : 3.2101404666900635
Epoch :   1, Step :  75, Val Loss : 3.2091281414031982
Epoch :   2, Step : 100, Val Loss : 3.202346086502075
Epoch :   2, Step : 125, Val Loss : 3.0961999893188477
Epoch :   3, Step : 150, Val Loss : 2.9404211044311523
Epoch :   4, Step : 175, Val Loss : 2.7973792552948
Epoch :   4, Step : 200, Val Loss : 2.6800122261047363
Epoch :   5, Step : 225, Val Loss : 2.5921859741210938
Epoch :   5, Step : 250, Val Loss : 2.4540796279907227
Epoch :   6, Step : 275, Val Loss : 2.3638625144958496
Epoch :   7, Step : 300, Val Loss : 2.2733516693115234
Epoch :   7, Step : 325, Val Loss : 2.2106523513793945
Epoch :   8, Step : 350, Val Loss : 2.167276620864868
Epoch :   8, Step : 375, Val Loss : 2.118539333343506
Epoch :   9, Step : 400, Val Loss : 2.079780340194702
Epoch :  10, Step : 425, Val Loss : 2.047388792037964
Epoch :  10, Step : 450, Val Loss : 2.013688087463379
Epoch :  11, Step : 

In [488]:
model_name = "selfModel.net"
torch.save(model.state_dict(), model_name)

In [489]:
model = CharModel(
    all_chars = all_characters,
    num_hidden = 512,
    num_layers = 3,
    drop_prob = 0.5,
    use_gpu = True
)
model

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [490]:
model.load_state_dict(torch.load(model_name))
model.eval()

CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [497]:
def predict_next_char(model, char, hidden = None, k = 1):

    encoded_text = model.encoder[char]

    encoded_text = np.array([[encoded_text]])

    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))

    inputs = torch.from_numpy(encoded_text)

    if model.use_gpu:
        inputs = inputs.cuda()

    hidden = tuple([state.data for state in hidden])

    lstm_out, hidden = model(inputs, hidden)

    probs = F.softmax(lstm_out, dim = 1).data

    if model.use_gpu:
        probs = probs.cpu()

    #print(probs)

    probs, index_positions = probs.topk(k)

    index_positions = index_positions.numpy().squeeze()

    probs = probs.numpy().flatten()

    probs = probs / probs.sum()

    char = np.random.choice(index_positions, p = probs)

    return model.decoder[char], hidden

In [498]:
def generate_text(model, size , seed = "The ", k = 1):

    if model.use_gpu:
        model.cuda()

    else:
        model.cpu()

    model.eval()

    output_chars = [c for c in seed]

    hidden = model.hidden_state(1)

    for char in seed:
        char, hidden = predict_next_char(model, char, hidden , k = k)

    output_chars.append(char)

    for i in range(size):

        char, hidden = predict_next_char(model, output_chars[-1], hidden, k = k)

        output_chars.append(char)

    return "".join(output_chars)

In [499]:
print(generate_text(model, 1000, seed = "The ", k=3 ))

The hour, the bory true, and me to have a
    trimmed and the self and sumper is not the fires of my fortune,  
    that I will see them that I had not be so so. The world she has strange strings to the
    true all to me and this is all the world. Why have you show this storant for
    the world, that thou hast sen her what is not a streeg.
  CELIA. This I hope that I am the world, sir, I have be this so much as the
    court of the wife of my fortunes. Wherefore, sir, I had never stown me. Therefore
    the woman is not to me. I have been the fine of heaven, and the means,  
    and they shall be their finger where in the wife was a poor of the sheet.
     [Aside] The selvices of my self we shall to the street to have the fool.
           And that the war to seem him.
                                                          Exeunt




SCENE VI.
The COUNTESS pOLLONE


  CENIA. Why, there's to the world with her with him is not any seem
    and many a mother's brood. Within him to my 